# Lab 3: 机器学习进阶技巧：特征工程和集成(Ensemble)模型

# Review

- Q1: [上节课]机器学习的三个关键步骤是什么？
- Q2: 怎么理解AUC？
- Q3: 怎么知道你是否过拟合了？
- Q4: 怎么进行parameter grid的选择？

## 2.1 Context 

### 2.1.1 元数据(Metadata)
| Column | Description | Type |
|--------|-------------|------|
| person_age | Age of the person | Float |
| person_gender | Gender of the person | Categorical |
| person_education | Highest education level | Categorical |
| person_income | Annual income | Float |
| person_emp_exp | Years of employment experience | Integer |
| person_home_ownership | Home ownership status (e.g., rent, own, mortgage) | Categorical |
| loan_amnt | Loan amount requested | Float |
| loan_intent | Purpose of the loan | Categorical |
| loan_int_rate | Loan interest rate | Float |
| cb_person_cred_hist_length | Length of credit history in years | Float |
| credit_score | Credit score of the person | Integer |
| loan_status (target variable) | Loan approval status: 1 = approved; 0 = rejected | Integer |

### 2.1.2 提供技术环境信息

- 请使用pandas， matplotlib， sklearn这三个python package（也可以在mdc文件中设置）

- 项目目录结构
```
|
|- data // 数据目录
|   |-- loan_data.csv
|- lab.ipynb
```

In [1]:
import matplotlib.pyplot as plt
# 设置中文字体
try:
    plt.rcParams['font.sans-serif'] = ['Heiti TC'] # 或者 'Arial Unicode MS', 'PingFang SC' 等
    plt.rcParams['axes.unicode_minus'] = False
except Exception as e:
    print(f"设置中文字体失败: {e}。标签可能显示不正确。")

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('data/loan_data.csv').query('previous_loan_defaults_on_file=="No"')
X = data.drop(['loan_status', 'previous_loan_defaults_on_file'], axis=1)
y = data['loan_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 特征工程

## 1.1 基础特征工程

特征工程一般有以下几种操作

- 异常值处理：（审慎）删除，以某个极值替代（例如年龄cap在100岁）

- 缺失值处理：删除【不推荐】，以均值或者众数填写

- One-hot Encoding：某些数据集会用数字来存储离散变量，再给一个code book（特别是社会科学的数据集，为了适配SPSS和Stata）。如果不做One-hot encoding会被当作数值变量处理

- 特征缩放：这避免某些特征的尺度对于其他特征的影响。比如把某个省的GDP和它的上年增长率放在一起，GDP的尺度会对于模型产生显著影响。特征缩放有两种常见方法，正态化或者range归一

- 高阶项（和交叉项）：虽然高阶项对于不同模型的好处并不完全相同，但是一般使用高阶项实际上是在做一个泰勒展开，去近似更复杂的function form（特别是对于SVM/Logit这样的线性模型）

这一节我们主要展示特征缩放和高阶项。提示词

```
# 代码逻辑
请构建一个pipeline，包含
1. 对于person_income,loan_amnt,credit_score的正态化scaling和对于person_age的range归一
2. 将上面两种变化后的变量 构建2阶polynomial
3. 对所有的categorical 做one-hot encoding，并构建2阶interaction only的polynomial
4. 使用decision tree classifer，criterion使用entropy，min_samples_split=2.基于auc_roc的5 fold crossvalidation 对max_depth，min_samples_split进行grid search，max depth的选择是[10,15,20],min_samples_split的选择是[40,60,80]
5. 汇报在train和test数据上的AUC，并将两个数据集的ROC画到一张图上

# 输出要求
1. 每个步骤写一个代码块
2. 使用详尽的注释
```

### 讨论
1. pipeline在其中起了什么作用？
2. 增加了特征后，模型表现变好了还是变差了？为什么？

# 1.2 Code Biz Insight

到现在为止，我们使用的都是工程技巧，而没有依赖任何业务知识。因为我们的tutorial 数据集本身比较干净。
在你们实际的数据集中，往往是原始数据，因此需要进行一些包含业务洞见的特征工程。比如，在贷款业务中，income/loan ratio 是一个非常重要的业务指标。这种除法指标（或者其他非线形可加）很难用多项式去近似准确复现这个特征

**回家作业 1** 请仅使用loan income ratio和*至多3个*你认为重要的业务指标重新进行构建决策树分类器。并思考这个新模型和我们刚才构建的模型有什么区别

# 2. Ensemble 模型

code insight的另一个重要方法是选择模型。在传统的统计学中（特别是在计量经济学中），选择用什么样的模型来描述数据生成过程是一个大学问。例如在经典的排队问题中，使用泊松模型还是负二次项模型大有讲究。如果未来你在硕士阶段选择了机器学习或者统计学，你会有大量时间花在研究似然函数（Likelihood function）上。

但是在Ensemble模型出现后，这个工作对于大部分场景而言已经不那么重要了。依靠算力大力飞砖被证明是性价比更高的选择。

## 2.1 Random Forest

提示词
```
将pipeline中的decision tree变成random forest，重新设计parameter grid search。数据集可能过拟合，请考虑这一点
```

**思考**：你现在的决策应该是什么？

## 2.2 特征重要性
使用Ensemble Model一个重要的好处是可以得到特征重要性。这使得特征工程可以更有针对性

**NOTE** 特征重要性和特征可解释性不一样。ensemble模型告诉你特征很重要，但是并不告诉你这是个正向因素还是个负向因素

**思考**：你现在的决策是什么？

## 2.3 Xgboost： Go-To Model Choice

提示词
```
# 代码逻辑
将pipeline中的模型变成xgboost，重新设计parameter grid search。
1. 数据集可能过拟合，请考虑这一点
2. 可以将并行计算设置为4个任务，tree数量为150，300，600
3. 输出训练集和测试集的AUC，并将ROC曲线画在一张图上；输出特征重要性

# 输出要求
1. 每个步骤写一个代码块
2. 使用详尽的注释
```

# 总结

这节课我们学习了：
- 如何使用pipeline来串联整个模型训练的routine，从而使得在预测时可以“屏蔽”再做一次特征工程的复杂性。通过封装（encapsulation）来隐藏复杂性这就是“engineering”的魅力。
- ensemble模型（特别是xgboost）能够有效对抗过拟合，且可以用算力来换准确性。因此传统classifer的关键就是**特征工程**

**回家作业**： 
1. 【必做】完成以loan/income ratio为主要特征的小模型，并思考和“大”模型的区别
2. 【选做】利用xgboost完成rainfall的完整pipeline，并提交一次Kaggle
